In [122]:
%run CFBmodel/import_modules.ipynb
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = 'MyjBpIrnMbCmsxXJD6zmiOCWqjB3p+7Icq5PRSrN82IXeOppkoEWybx1SmRryxT+'
configuration.api_key_prefix['Authorization']='Bearer'
api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))
# game_id=game_id  season_type=season_type, team=team, home=home, away=away, conference=conference)

In [123]:
con=sqlite3.connect('CFBModel/collegeFootball.db')
cursor_obj= con.cursor()
games = pd.read_sql("""SELECT * FROM Games23""", con)

In [128]:
games.id

0       401550883
1       401525434
2       401540199
3       401520145
4       401532392
          ...    
3594    401539484
3595    401539477
3596    401539478
3597    401539480
3598    401520445
Name: id, Length: 3599, dtype: int64

In [129]:
api_instance.get_advanced_box_score(game_id=401550883)

{'players': None, 'teams': None}

In [ ]:
empty_games_list=[]
def parse_data(game_id):
    data = api_instance.get_advanced_box_score(game_id=game_id)
    # Check if data is None
    if ((data.players is None) & (data.teams is None)):
        empty_games_list.append(game_id)
        return pd.DataFrame(), pd.DataFrame()  # Return empty DataFrames

    combined_players = []
    combined_teams = []

    # Parse players
    for player in data.players.ppa:
        player_info = {
                'game_id': game_id,  # Add game_id to player info
                'player': player.player,
                'position': player.position,
                'team': player.team,
                'average_passing': player.average.passing,
                'average_rushing': player.average.rushing,
                'cumulative_passing': player.cumulative.passing,
                'cumulative_rushing': player.cumulative.rushing,
                'total_average': player.average.total,
                'total_cumulative': player.cumulative.total
            }
        combined_players.append(player_info)

        # Parse teams
    for team in data.teams.cumulative_ppa:
        team_info = {
                'game_id': game_id,  # Add game_id to team info
                'team': team.team,
                'plays': team.plays,
                'overall_quarter1': team.overall.quarter1,
                'overall_quarter2': team.overall.quarter2,
                'overall_quarter3': team.overall.quarter3,
                'overall_quarter4': team.overall.quarter4,
                'overall_total': team.overall.total,
                'passing_quarter1': team.passing.quarter1,
                'passing_quarter2': team.passing.quarter2,
                'passing_quarter3': team.passing.quarter3,
                'passing_quarter4': team.passing.quarter4,
                'passing_total': team.passing.total,
                'rushing_quarter1': team.rushing.quarter1,
                'rushing_quarter2': team.rushing.quarter2,
                'rushing_quarter3': team.rushing.quarter3,
                'rushing_quarter4': team.rushing.quarter4,
                'rushing_total': team.rushing.total
            }
        combined_teams.append(team_info)

        # Create DataFrames from the lists of dictionaries
    players_df = pd.DataFrame(combined_players)
    teams_df = pd.DataFrame(combined_teams)

    return players_df, teams_df, empty_games_list
def get_box_scores(game_ids):
    results = [parse_data(game_id) for game_id in game_ids]

    # Split the results into players and teams DataFrames
    players_data = [result[0] for result in results]
    teams_data = [result[1] for result in results]

    # Concatenate all DataFrames
    combined_players_df = pd.concat(players_data, ignore_index=True) if players_data else pd.DataFrame()
    combined_teams_df = pd.concat(teams_data, ignore_index=True) if teams_data else pd.DataFrame()

    return combined_players_df, combined_teams_df

combined_players_df_2023, combined_teams_df_2023 = get_box_scores(games.id)


In [ ]:
con=sqlite3.connect('CFBModel/collegeFootball.db')
cursor_obj= con.cursor()
cursor_obj.execute("DROP TABLE IF EXISTS playerBoxScore23")
combined_players_df_2023.to_sql(name='playerBoxScore23',con=con)

con=sqlite3.connect('CFBModel/collegeFootball.db')
cursor_obj= con.cursor()
cursor_obj.execute("DROP TABLE IF EXISTS teamBoxScore23")
combined_teams_df_2023.to_sql(name='teamBoxScore23',con=con)

In [ ]:
con.commit()
con.close()